In [1]:
# libraries required 
import torch
import torchvision 
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

**Basic Auto-grad**

In [2]:
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)
# set requires_grad=True to track computation with it

In [4]:
y = w*x+b
y.backward() # backpropagation

In [5]:
print(x.grad)  # dy/dx = w
print(w.grad)  # dy/dw = x
print(b.grad)  # dy/db = 1

tensor(4.)
tensor(2.)
tensor(2.)


**Auto-grad**

In [7]:
#random vector for x and y
x = torch.randn(10,3)
y = torch.rand(10,2)

In [8]:
linear =  nn.Linear(3,2) # fully connected layer
print("w: ", linear.weight) # print the weight parameters
print("b: ", linear.bias)   # print the bias parameters

w:  Parameter containing:
tensor([[-0.2819,  0.5300, -0.2647],
        [-0.2197,  0.4136,  0.1172]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.1443, -0.2901], requires_grad=True)


In [9]:
criterion = nn.MSELoss() # define the loss 
optimizer = torch.optim.SGD(linear.parameters(),lr =0.01) # optimization algo

In [10]:
pred = linear(x) # forward pass

In [11]:
loss = criterion(pred,y) # calculate loss
print("loss: ",loss.item())

loss:  0.5926915407180786


In [12]:
loss.backward() #backpropagation

In [13]:
print("dl/dw: ",linear.weight.grad) # print the gradients 
print("dl/db: ",linear.bias.grad)  

dl/dw:  tensor([[ 0.1656,  0.3095, -0.1630],
        [ 0.4973,  0.4723,  0.0627]])
dl/db:  tensor([-0.4414, -0.8597])


In [14]:
optimizer.step() # change the learned paramters using SGD 

In [15]:
pred = linear(x) # forward pass
loss = criterion(pred,y) # find loass after one step optimization
print("loss after 1 step optimizer: ", loss.item())

loss after 1 step optimizer:  0.5772436857223511


**Loading data from numpy**

In [21]:
x = np.array([[1,2],[3,4]])

In [22]:
y = torch.from_numpy(x)
# Convert back to numpy
# z = y.numpy()

In [23]:
train_dataset = torchvision.datasets.CIFAR10(root='data/data/',train=True,download=True,transform=transforms.ToTensor())

**Load dataset from disk**

In [25]:
# load data from the disk
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                          batch_size=64,
# define the iterator                                          shuffle=True)
data_itr = iter(train_loader)

In [26]:
#read one batch using the iterator
images, labels = data_itr.next()

In [35]:
#Custom dataset to load the batch easily using the train_loader
class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self):
        # initialize file paths or a list of the file names.
        pass
    def __getitem__(self,index):
        """
            Step 1: Read data from the file 
            Step 2: Transform the data point
            Step 3: Return the data pair

        """
        
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0


In [36]:
# laod teh custom dataset using the data loader 
custom_dataset = CustomDataSet()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                          batch_size=64,
                                          shuffle=True)

**Load the pretrain model**

In [39]:
# Pre-train model
resnet = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/vedic/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:20<00:00, 2271663.78it/s]


In [44]:
# Replace the top layer for finetuning 
for par in resnet.parameters():
    par.requires_grad = False
resnet.fc = nn.Linear(resnet.fc.in_features,100)

In [46]:
#random image 
images = torch.randn(64,3,224,224)

In [48]:
# test the pretrain model on random image
outputs = resnet(images)
print(outputs.size())

torch.Size([64, 100])


In [51]:
# Save and load model entire model

torch.save(resnet,'data/models/model.ckpt')
model = torch.load('data/models/model.ckpt')

# save and load the model parameters --RECOMMENDED
torch.save(resnet.state_dict(),'data/models/params.ckpt')
model = resnet.load_state_dict(torch.load('data/models/params.ckpt'))